[Reference](https://medium.com/geoblinktech/fastapi-with-api-versioning-for-data-applications-2b178b0f843f)

# API versioning with FastAPI


In [ ]:
from fastapi import FastAPI
from fastapi_versioning import VersionedFastAPI, version

app = FastAPI(title="My App")


@app.get("/greet")
@version(1, 0)
def greet_with_hello():
    return "Hello"


@app.get("/greet")
@version(1, 2)
def greet_with_hi():
    return "Hi"

@app.get("/goodbye")
@version(1, 1)
def say_goodbye():
    return “bye”

@app.get("/foo")
def get_foo():
    return “foo”


app = VersionedFastAPI(app, default_api_version=(1, 2))

In [ ]:
app = FastAPI(
title="My Item App”,
exception_handlers={
       500: internal_error_exception_handler,  # Uncontrolled internal server errors (e.g. raised by FastAPI's middlewares)
       RequestValidationError: request_validation_exception_handler,  # Custom data validation error
   }
)

In [ ]:
async def request_validation_exception_handler(request: Request, exc: RequestValidationError):
    return PlainTextResponse(str(exc), status_code=400)

In [ ]:
class APIVersion:

   def __init__(self, major_version, minor_version):
       self._major_version = major_version
       self._minor_version = minor_version

   def to_tuple(self) -> Tuple[int, int]:
       return self._major_version, self._minor_version

   def to_str(self) -> str:
       return f"v{self._major_version}_{self._minor_version}"

In [ ]:
def version_app(
        app: FastAPI,
        default_api_version: APIVersion,
        exception_handlers: Optional[Dict[Union[int, Type[Exception]], Callable]],
        **kwargs
):
    app = VersionedFastAPI(
        app,
        version=default_api_version.to_str(),  # Version that appears at the top of the API docs
        default_version=default_api_version.to_tuple(),  # Version at which unversioned endpoints start to be available
        exception_handlers=exception_handlers,
        **kwargs
    )

    # Hack: Register exception handlers in all mounted subapps
    # We need this workaround because fastapi-versioning is not passing them downstream to sub-apps by default
    mounted_routes = [route for route in app.routes if isinstance(route, Mount)]

    if exception_handlers is not None:
        for mounted_route in mounted_routes:
            for exc, exc_handler in exception_handlers.items():
                mounted_route.app.add_exception_handler(exc, exc_handler)

    return app

In [ ]:
LATEST_API_VERSION = APIVersion(major_version=1, minor_version=1)

# Create app object and add routes
app = FastAPI(title="My Item App")
app.include_router(unversioned_router)
app.include_router(router_v1_0)
app.include_router(router_v1_1)


exception_handlers = {
        500: internal_error_exception_handler,  # Uncontrolled internal server errors (e.g. raised by FastAPI's middlewares)
        CustomServiceError: internal_error_exception_handler,  # Controlled internal server errors
        RequestValidationError: request_validation_exception_handler,  # Custom data validation error
    }

app = version_app(app, default_api_version=LATEST_API_VERSION, exception_handlers=exception_handlers)


In [ ]:
async def request_validation_exception_handler(request: Request, exc: RequestValidationError) -> JSONResponse:
    return JSONResponse(
        status_code=HTTP_422_UNPROCESSABLE_ENTITY,
        content={
                "error_name": exc.__class__.__name__,
                "message": jsonable_encoder(exc.errors()),
        },
    )